In [0]:
!pip install git+https://github.com/rcmalli/keras-vggface.git ### GETTING FACE FEATURE EXTRACTION BY VGG_FACE IMPLEMENTED IN KERAS.

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-7dzcl_5c
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-7dzcl_5c
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=11e44db891387fe8f94f55ac43053d911887341447fe92ce8f933e033863d5cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-ey_z6fpq/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import keras.layers
from tqdm import tqdm
import cv2
import dlib
import os

Using TensorFlow backend.


In [0]:
%cd /content/

/content


In [0]:
!mkdir face

mkdir: cannot create directory ‘face’: File exists


In [0]:

!7z e /content/drive/My\ Drive/facess.7z -o/content/face


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/My Drive/                                  1 file, 24971361 bytes (24 MiB)

Extracting archive: /content/drive/My Drive/facess.7z
--
Path = /content/drive/My Drive/facess.7z
Type = 7z
Physical Size = 24971361
Headers Size = 35420
Method = LZMA2:24
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     /content/face/0.jpg
  Size:     7109 bytes (7 KiB)
  Modified: 2019-10-04 08:22:37
with the file from archive:
  Path:     facess/0.jpg
  Size:     7109 bytes (7 KiB)
  Modified: 2019-10-04 08:22:37
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? Q

Archives with Errors: 1



Break signaled


In [0]:
metadata = pd.read_csv("/content/drive/My Drive/datasetss/vox1_meta.csv",delimiter = "\t")
metadata.head()

,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,id10001,A.J._Buckley,m,Ireland,dev
1,id10002,A.R._Rahman,m,India,dev
2,id10003,Aamir_Khan,m,India,dev
3,id10004,Aaron_Tveit,m,USA,dev
4,id10005,Aaron_Yoo,m,USA,dev


In [0]:
%cd /content
!ls

/content
drive  face  sample_data


In [0]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [0]:
from keras_vggface.vggface import VGGFace
vgg_features = VGGFace(include_top=True, input_shape=(224, 224, 3))
vgg_features.layers.pop()
vgg_features.layers.pop()
vgg_features.outputs = [vgg_features.layers[-1].output]
vgg_features.layers[-1].outbound_nodes = []

vgg_features.summary()# vgg_features.layers.pop()








Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128

In [0]:
! pwd

/content/drive/My Drive


In [0]:
def detect( img_path):
    """
    
    THIS FUNCTION DETECTS THE FACE IN GREY IMAGE AND CROP IT THEN PREDICT ITS  FACE FEATURE
    
    PAPAMETER:
    img_path: str; PATH OF IMAGE
    
    RETURNS:
    FACE_FEATURE: np.array; PREDICTED ARRAY OF SIZE (1,2048)
    FRAME_CROP: np array; CROPED IMAGE
    
    """
    frame = cv2.imread(img_path)
    cnn_face_detector = dlib.cnn_face_detection_model_v1('./mmod_human_face_detector.dat')
    faces_cnn = cnn_face_detector(frame, 1)
    
    for face in faces_cnn:
       
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
    try:
      frame = frame[x:x + w,y:y+h]
      frame = cv2.resize(frame,(224,224)).reshape(1,224,224,3)
      return vgg_features.predict(frame), frame
    except:
      print(img_path)
      return np.zeros((1,4096)),np.zeros((1,224,224,3))

In [0]:
Y = np.ones((3633,4096))
for i in range(3633):
  img_path = "/content/face/" + str(i) + ".jpg"
  if os.path.exists(img_path):
    Y[i,:],ac_frame = detect(img_path)
    cv2.imwrite("face/"+str(i)+".jpg",ac_frame)
  else:
    print(i)
  if i/500 == 0:
    print("#",end = "")


In [0]:
np.save("Picture_feature.npy",Y)